**Imports**

In [173]:
# ---------------------------------------------- Imports ----------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

**Constantes**

In [174]:
L = 0.230
R = 0.035
ROBOT = "kobuki"
LASER = "fastHokuyo"

robot_path = []
obstacle_points = []

**Init**

In [175]:
# ---------------------------------------------- Init ----------------------------------------
try:
    client = RemoteAPIClient()
    sim = client.require('sim')
except:
    print('Error connection')

**Leitura do Sensor**

In [176]:
def readSensorData(range_data_signal_id="hokuyo_range_data", 
                    angle_data_signal_id="hokuyo_angle_data"):
    
    signalName = sim.waitForSignal(range_data_signal_id)
    
    string_range_data = sim.getStringSignal(range_data_signal_id)

    string_angle_data = sim.getStringSignal(angle_data_signal_id)

    # verifique se ambos os dados foram obtidos corretamente
    if string_range_data != None and string_angle_data != None:
        # descompacte dados de mensagens de alcance e sensor
        raw_range_data = sim.unpackFloatTable(string_range_data)
        raw_angle_data = sim.unpackFloatTable(string_angle_data)

        return raw_range_data, raw_angle_data

    # retornar nenhum caso nada tenha sido obtido do simulador
    return None

**Cálculo Matriz de Rotação**

In [177]:
def get_rotation_matrix(thetaZ):
    return np.array([[  np.cos(thetaZ), -np.sin(thetaZ), 0 ],
                      [ np.sin(thetaZ),  np.cos(thetaZ), 0 ],
                      [ 0            ,  0            , 1 ]])

def get_rotation_matrix_laser_world():
    objectHandle_RP = sim.getObject("/" + ROBOT)
    origin_RP = np.array(sim.getObjectPosition(objectHandle_RP, 
                                        sim.handle_world))

    objectHandle_Laser = sim.getObject("/" + LASER)
    origin_Laser = np.array(sim.getObjectPosition(objectHandle_Laser, 
                                        sim.handle_world))
    p_Laserorg = origin_Laser - origin_RP
    eulerAngles_Laser = sim.getObjectOrientation(objectHandle_Laser, 
                                                objectHandle_RP)

    aux = np.array([0, 0, 0, 1])
    rotationMatrix_Laser = get_rotation_matrix(eulerAngles_Laser[2])
    transformationMatrix_Laser = np.column_stack((rotationMatrix_Laser, p_Laserorg))
    transformationMatrix_Laser = np.row_stack((transformationMatrix_Laser, aux))

    #Matriz de Transformação Robô -> mundo
    origin_World = np.array([0,0,0])
    p_RPorg = origin_RP - origin_World
    eulerAngles_RP = sim.getObjectOrientation(objectHandle_RP, 
                                                sim.handle_world)
    rotationMatrix_RP = get_rotation_matrix(eulerAngles_RP[2])
    transformationMatrix_World = np.column_stack((rotationMatrix_RP, p_RPorg))
    transformationMatrix_World = np.row_stack((transformationMatrix_World, aux))
    
    #Matriz de Transformação Laser -> Mundo
    return transformationMatrix_Laser @ transformationMatrix_World

**Campos Potenciais**

In [178]:
#Calcula a força de atração 
def att_force(q, goal, k=5):
    f = k*(goal - q)
    return f

#Calcula a força de repulsão
def rep_force(q, obs, R=1.5, krep=.005):
    Frep_total = np.array([0.0, 0.0])
    for obstacle in obs:
        v = q[0:2] - obstacle
        d = np.linalg.norm(v) # Calcula a distância entre o robô e o obstáculo

        # Calcula a força de repulsão apenas se a distância for menor que R
        if (d < R):  
            rep = (1/d**2)*((1/d)-(1/R))*(v/d) 
            Frep_total += rep

    return krep*Frep_total

**Occupancy Grid**

In [ ]:
#Leitura -> Célula
def convert_robot_read_to_laser(pos_robot):
    matrix = np.array([])

def convert_laser_to_cell():
    return laser_position/cell_size

**Navegação**

In [179]:
def navigation(qgoal):
    robotHandle = sim.getObject("/" + ROBOT)  
    robotLeftMotorHandle = sim.getObject("/" +'kobuki_leftMotor')
    robotRightMotorHandle = sim.getObject("/" +'kobuki_rightMotor')

    maxv = 0.2 #LIMITADORES DA VELOCIDADE LINEAR
    maxw = np.deg2rad(45) #LIMITADORES DA VELOCIDADE ANGULAR

    rho = np.inf
    random_force = 0.1
    max_sensor_range = 5

    while rho > .05:
        obstacle = []

        #Configuração do robo
        robotPos = sim.getObjectPosition(robotHandle, sim.handle_world)
        robotPos = robotPos[0:2]
        robot_path.append(robotPos)
        robotOri = sim.getObjectOrientation(robotHandle, sim.handle_world)        
        robotConfig = np.array([robotPos[0], robotPos[1], robotOri[2]])

        raw_range_data, raw_angle_data = readSensorData()
        laser_data = np.array([raw_angle_data, raw_range_data]).T

        #Transformação dos dados do meu laser para o mundo
        transformationMatrix_LaserWorld = get_rotation_matrix_laser_world()

        for i in range(len(laser_data)):
            ang, dist = laser_data[i] #pega os valores de angulo e distância

            if (max_sensor_range - dist) > 0.1:
                x = dist * np.cos(ang) #meu x
                y = dist * np.sin(ang) #meu y
                point = np.array([x,y,0,1])

                if len(transformationMatrix_LaserWorld) != 0:
                    point = transformationMatrix_LaserWorld @ point
                    obstacle.append(point[0:2])
                    obstacle_points.append(point)    
        
        Fatt = att_force(robotPos, qgoal)
        Frep = rep_force(robotPos, obstacle)

        random_force_vector = random_force * np.random.rand(2)
        Ft = Fatt + Frep + random_force_vector
        Ft_x = Ft[0]
        Ft_y = Ft[1]

        # Apenas para interromper o loop
        rho = np.sqrt(Ft_x**2 + Ft_y**2)
        
        # Formula De Luca e Oriolo para calcular a velocidade linear e angular.
        kr = 0.05
        kt = 0.1
        v = kr*(Ft_x*np.cos(robotConfig[2]) + Ft_y*np.sin(robotConfig[2]))
        w = kt*(np.arctan2(Ft_y,Ft_x) - robotConfig[2])
                
        # Limit v,w to +/- max
        v = max(min(v, maxv), -maxv)
        w = max(min(w, maxw), -maxw)        
        
        vr = ((1.0*v) + (w*L))/(2.0*R)
        vl = ((1.0*v) - (w*L))/(2.0*R)
        sim.setJointTargetVelocity(robotRightMotorHandle, vr)
        sim.setJointTargetVelocity(robotLeftMotorHandle, vl)

    sim.setJointTargetVelocity(robotRightMotorHandle, 0)
    sim.setJointTargetVelocity(robotLeftMotorHandle, 0)



**MAIN**

In [180]:
# TESTE 1
qgoals = []
qgoals.append([1.5, 3])
qgoals.append([3.4, 4])
qgoals.append([0.4, 4])
qgoals.append([1.0, 0])
qgoals.append([0.5, -4.5])

In [181]:
sim.startSimulation()

reference_frame_handle = sim.getObject("/" +'ReferenceFrame')

for qgoal in qgoals:
        reference_frame_position = qgoal.copy()
        reference_frame_position.append(0)
        sim.setObjectPosition(reference_frame_handle, reference_frame_position)
        navigation(np.array(qgoal))

sim.stopSimulation()
# Após o término da simulação
# Plotando o caminho do robô
robot_path = np.array(robot_path)
plt.plot(robot_path[:, 0], robot_path[:, 1], '-r', label='Caminho do Robô')

# Plotando os obstáculos
# for point in obstacle_points:
#         plt.plot(point[0], point[1], 'o', color='b')
obstacle_points = np.array(obstacle_points)
plt.plot(obstacle_points[:, 0], obstacle_points[:, 1], 'o', color='b', label='Obstáculos')


plt.xlabel('X')
plt.ylabel('Y')
plt.title('Caminho do Robô e Obstáculos')
plt.legend()
plt.grid(True)
plt.axis('equal')

(-5.6424067781568334, 6.024320934221436, -6.206585096391393, 5.936908673041858)

Error in callback <function flush_figures at 0x000002A31CFBEE80> (for post_execute):


KeyboardInterrupt: 